In [19]:
import pandas as pd

import numpy as np
import string as s
import seaborn as sns
import math
import tables
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import style, colors
from itertools import groupby
import matplotlib.patches as patches
from matplotlib import gridspec
import scipy.stats as stats
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster, fclusterdata, leaves_list
from scipy.spatial.distance import pdist
from scipy.ndimage.interpolation import shift
# matplotlib.style.use('ggplot')
# %matplotlib inline
matplotlib.rcParams['figure.figsize'] = (12, 6)
# matplotlib.rcParams['figure.facecolor'] = 'white'
import mpl_toolkits.mplot3d.axes3d as p3
from sklearn.cluster import AgglomerativeClustering
from sklearn.datasets.samples_generator import make_swiss_roll
matplotlib.style.use('ggplot')
%matplotlib inline

questions for Misha:
repressive marks: H3K9me3 and H3K27me3, 
broad marks: H3K4me1, H3K9me3, H3K27me3, H3K36me3
1. for 6 marks: marks = "H3K4me1","H3K4me3","H3K9me3","H3K27me3","H3K36me3","H3K27ac", what regions am i supposed to look at. for example, for H3K4me3 and H3K4me1 and H3K27ac and H3K27me3(negative mark means gene not expressed), , I can look at putative promoter regions.
H3K9me3(repressive mark ), complex repetative  regions. leave for later, mark for repetative regions and genes important for development, 1 million regions, bin genome to 10kb bins remove 0 values and cluster on the whole genome   
H3K36me3, use gene body. ask file from Misha, 
H3K4me1 and H3K27ac() can cluster on enhancer and txn binding regions.

2. confirm my workflow are correct:
1) intersect bdg file with hg19v69_genes.TSS_2000.pc.bed
2) sum bdg values for each putative promoters and divide by region length (4000) to get average signal values. i should have integrated on the various bins, (end-start +1)*coverage. see if 0 or 1 based. also Alessia requested using jaccard value, basically i will need a segmenation bed file telling the regions are enriched and then get the jaccard value while overlap enriched regions and putative promoter regions or gene body. 
3) cluster 


chromatin states: 100+ histone modifictions
example:
H3K4me3: promoters
H3K4me1: enhancers
H3K9me3: repressive
H3K27me3:repressive
H3K36me3: transcribed


cluster H3k36me3, transcribed mark, cluster on gene body, use this file:/home/szong/projects/resource/hg19v69_genes_body.pc

# intersect pileup file with gene body file, need to run on gphost, insufficient memory on local machinek chr suffix is not an issue

while read i;do /home/rcorbett/aligners/bedtools/BEDTools-Version-2.15.0/bin/bedtools intersect -a ../hg19v69_genes_body.pc -b /projects/seqdev/DA-77/new_pipeline_treatment/step2/$i -wa -wb > $i.intersected;done < ../all_H3K36me3_treat_pileup_bdg_files_names.txt

In [20]:
wkdir = '/projects/trans_scratch/validations/workspace/szong/Cervical/chipseq/'

get meta data about library ids, hiv status, histology, total reads sequenced etc

In [21]:
f = '/home/szong/projects/META_2018/Master_Sample_table_14X292_Feb27_2018_sz.xls'
meta = pd.read_excel(f, sheetname='HTMCP CaCx epi')
meta.head(2)

,Order,Biospecimen ID,HIV status,Epigenome modifier mutation status,H3K4me1 library ID,H3K4me3 library ID,H3K9me3 library ID,H3K27me3 library ID,H3K36me3 library ID,H3K27ac library ID,...,H3K4me3 % aligned,H3K9me3 % aligned,H3K27me3 % aligned,H3K36me3 % aligned,H3K27ac % aligned,input DNA control % aligned,internal sample ID,pool ID,Notes,# lanes 7 PE
0,1,HTMCP-03-06-02007-01A,negative,mutated,A84624,A84625,A84626,A84627,A84628,A84629,...,98.13,93.1,97.93,98.57,98.10,96.26,NaN,IX5978,NaN,1
1,2,HTMCP-03-06-02026-01A,negative,mutated,A84645,A84646,A84647,A84648,A84649,A84650,...,97.68,93.89,97.84,98.17,97.86,95.56,NaN,IX5981,NaN,1


In [22]:
meta = meta[['Biospecimen ID', 'H3K4me1 library ID',
       'H3K4me3 library ID', 'H3K9me3 library ID', 'H3K27me3 library ID',
       'H3K36me3 library ID', 'H3K27ac library ID',
       'input DNA control  library ID',  'H3K4me1 total reads', 'H3K4me3  total reads',
       'H3K9me3 total reads', 'H3K27me3 total reads', 'H3K36me3 total reads',
       'H3K27ac total reads', 'input DNA control  total reads']].dropna()

In [23]:
meta.head(2)

,Biospecimen ID,H3K4me1 library ID,H3K4me3 library ID,H3K9me3 library ID,H3K27me3 library ID,H3K36me3 library ID,H3K27ac library ID,input DNA control library ID,H3K4me1 total reads,H3K4me3 total reads,H3K9me3 total reads,H3K27me3 total reads,H3K36me3 total reads,H3K27ac total reads,input DNA control total reads
0,HTMCP-03-06-02007-01A,A84624,A84625,A84626,A84627,A84628,A84629,A84630,78611726,60179836,76557498,90312752.0,106610232.0,80676254.0,86539886.0
1,HTMCP-03-06-02026-01A,A84645,A84646,A84647,A84648,A84649,A84650,A84651,85566812,74487212,93042922,89559600.0,87905340.0,84986034.0,88575282.0


In [24]:
meta['patient'] = meta['Biospecimen ID'].apply(lambda x: '-'.join(['HTMCP-03-06', x.split('-')[3]]))

In [25]:
meta = meta.drop('Biospecimen ID', axis=1)

In [26]:
meta.head(2)

,H3K4me1 library ID,H3K4me3 library ID,H3K9me3 library ID,H3K27me3 library ID,H3K36me3 library ID,H3K27ac library ID,input DNA control library ID,H3K4me1 total reads,H3K4me3 total reads,H3K9me3 total reads,H3K27me3 total reads,H3K36me3 total reads,H3K27ac total reads,input DNA control total reads,patient
0,A84624,A84625,A84626,A84627,A84628,A84629,A84630,78611726,60179836,76557498,90312752.0,106610232.0,80676254.0,86539886.0,HTMCP-03-06-02007
1,A84645,A84646,A84647,A84648,A84649,A84650,A84651,85566812,74487212,93042922,89559600.0,87905340.0,84986034.0,88575282.0,HTMCP-03-06-02026


In [27]:
# add in hiv status and histology
f22 ='/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients/mutation_load_clinic.txt'
df22 = pd.read_csv(f22, sep='\t')
df22 = df22[['patient', 'HIV_status', 'Putative_histology', 'strelka_mutations', 'HPV_status']]
# df22 = df22.set_index('patient')#.stack().reset_index()
df22.head(2)

,patient,HIV_status,Putative_histology,strelka_mutations,HPV_status
0,HTMCP-03-06-02001,Positive,Squamous,26974,Positive
1,HTMCP-03-06-02002,Negative,Squamous,16661,Positive


In [28]:
df_meta = pd.merge(meta, df22, left_on='patient', right_on='patient')
df_meta.head(2)

,H3K4me1 library ID,H3K4me3 library ID,H3K9me3 library ID,H3K27me3 library ID,H3K36me3 library ID,H3K27ac library ID,input DNA control library ID,H3K4me1 total reads,H3K4me3 total reads,H3K9me3 total reads,H3K27me3 total reads,H3K36me3 total reads,H3K27ac total reads,input DNA control total reads,patient,HIV_status,Putative_histology,strelka_mutations,HPV_status
0,A84624,A84625,A84626,A84627,A84628,A84629,A84630,78611726,60179836,76557498,90312752.0,106610232.0,80676254.0,86539886.0,HTMCP-03-06-02007,Negative,Adeno,2958282,Negative
1,A84645,A84646,A84647,A84648,A84649,A84650,A84651,85566812,74487212,93042922,89559600.0,87905340.0,84986034.0,88575282.0,HTMCP-03-06-02026,Negative,Adeno,1252217,Negative


In [29]:
of = '/projects/trans_scratch/validations/workspace/szong/Cervical/chipseq/chipseq_meta.txt'
df_meta.to_csv(of, sep='\t', index=False)

In [30]:
category_track = df_meta[['patient', 'HIV_status', 'Putative_histology', 'HPV_status']]
numeric_track = df_meta[['patient', 'strelka_mutations', ]]

In [31]:
category_track.head(2)

,patient,HIV_status,Putative_histology,HPV_status
0,HTMCP-03-06-02007,Negative,Adeno,Negative
1,HTMCP-03-06-02026,Negative,Adeno,Negative


In [32]:
numeric_track.head(2)


,patient,strelka_mutations
0,HTMCP-03-06-02007,2958282
1,HTMCP-03-06-02026,1252217


In [33]:
numeric_track['log_strelka_mutations'] = np.log10(numeric_track['strelka_mutations'])

/home/szong/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [34]:
numeric_track = numeric_track.drop('strelka_mutations', axis=1)

In [35]:
numeric_track.head(2)

,patient,log_strelka_mutations
0,HTMCP-03-06-02007,6.47104
1,HTMCP-03-06-02026,6.09768


read in Alessia's epigenetic gene categorization file

In [36]:
f4 = '/projects/trans_scratch/validations/workspace/szong/Cervical/chipseq/categorisation_of_epigentic_modifiers_from_Alessia_v3.xls'
df4 = pd.read_excel(f4, sheetname='categorisation')
df4.head(2)

,HIV neg,neg samples with mutations,HIV positive,pos samples with mutation,total,Category_1,Category_2,markes /regiones affected,comments
0,MLL3,9,MLL3,6,15,Writer,compass complex,H3k4me1,NaN
1,MLL2,7,MLL2,5,12,Writer,compass complex,H3K4me3,NaN


In [37]:
df41 = df4[['HIV neg', 'Category_1', 'Category_2']].set_index('HIV neg')
df41.head(2)

,Category_1,Category_2
HIV neg,,
MLL3,Writer,compass complex
MLL2,Writer,compass complex


In [38]:
df41.Category_1.unique()

array(['Writer', 'Remodeller', 'Eraser', 'Reader', nan], dtype=object)

In [39]:
df41.Category_2.value_counts()

Polycomb complex                                                              10
compass complex                                                                9
INO80/SWR remodelling complexes                                                9
ISWI remdelling complexes                                                      7
HDAC complex                                                                   7
GNAT family HAT complexes                                                      7
NuRD complex                                                                   5
MYST family HAT complexes                                                      5
SWI/SNF remodeling complex                                                     5
SEC (Super Elongation) Complex                                                 4
EP300/CBP HAT complexes                                                        3
TFIID basal transcription factor complex or PCAF histone acetylase complex     3
chromatin remodelling       

In [40]:
df5 = pd.read_excel(f4, sheetname='sample_mut_details_sz', index_col='patient')
df5.head(2)

,HIV status,ARID1A,ATRX,BAZ1A,BAZ1B,BAZ2B,BRD2,BRD3,BRD4,BRD7,...,TBL1X,TFPT,TRIM33,UBE2B,TRRAP,WHSC1L1,YEATS2,YY1,ZMYND11,ZMYND8
patient,,,,,,,,,,,,,,,,,,,,,
HTMCP-03-06-02007,HIV negative,1,1,1,0,1,0,0,1,1,...,0,0,0,0,1,0,0,1,0,1
HTMCP-03-06-02026,HIV negative,1,1,0,1,1,0,1,0,1,...,0,0,1,0,0,0,0,0,1,0


In [41]:
df51 = df5[df5.columns[1:]].T

df51.head(2)

patient,HTMCP-03-06-02007,HTMCP-03-06-02026,HTMCP-03-06-02063,HTMCP-03-06-02214,HTMCP-03-06-02042,HTMCP-03-06-02332,HTMCP-03-06-02185,HTMCP-03-06-02012,HTMCP-03-06-02196,HTMCP-03-06-02203,...,HTMCP-03-06-02076,HTMCP-03-06-02113,HTMCP-03-06-02230,HTMCP-03-06-02170,HTMCP-03-06-02152,HTMCP-03-06-02235,HTMCP-03-06-02195,HTMCP-03-06-02261,HTMCP-03-06-02147,HTMCP-03-06-02267
ARID1A,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ATRX,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
df6 = pd.merge(df51, df41, left_index=True, right_index=True)

df6.head(2)

,HTMCP-03-06-02007,HTMCP-03-06-02026,HTMCP-03-06-02063,HTMCP-03-06-02214,HTMCP-03-06-02042,HTMCP-03-06-02332,HTMCP-03-06-02185,HTMCP-03-06-02012,HTMCP-03-06-02196,HTMCP-03-06-02203,...,HTMCP-03-06-02230,HTMCP-03-06-02170,HTMCP-03-06-02152,HTMCP-03-06-02235,HTMCP-03-06-02195,HTMCP-03-06-02261,HTMCP-03-06-02147,HTMCP-03-06-02267,Category_1,Category_2
ARID1A,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Remodeller,SWI/SNF remodeling complex
ATRX,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Remodeller,SWI/SNF family


In [43]:
def categorize_cols(df, col):
    col_values = df[col].unique()
    lst = []
    for col_value in col_values:
        s = df.query('{0} == "{1}"'.format(col, col_value)).sum()
        lst.append(s)
    dfo = pd.DataFrame(lst)
    dfo.index = col_values
    dfo = dfo.drop([col], axis=1)
    return dfo

In [44]:
df8 = categorize_cols(df6, 'Category_1').drop('Category_2', axis=1)

df8.head(2)

,HTMCP-03-06-02001,HTMCP-03-06-02002,HTMCP-03-06-02006,HTMCP-03-06-02007,HTMCP-03-06-02012,HTMCP-03-06-02020,HTMCP-03-06-02026,HTMCP-03-06-02036,HTMCP-03-06-02040,HTMCP-03-06-02042,...,HTMCP-03-06-02266,HTMCP-03-06-02267,HTMCP-03-06-02268,HTMCP-03-06-02320,HTMCP-03-06-02330,HTMCP-03-06-02332,HTMCP.03.06.02037,HTMCP.03.06.02046,HTMCP.03.06.02150,HTMCP.03.06.02174
Remodeller,0.0,0.0,2.0,16.0,1.0,1.0,9.0,0.0,2.0,2.0,...,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
Reader,1.0,0.0,0.0,21.0,1.0,1.0,7.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0


In [45]:
df9 = categorize_cols(df6, 'Category_2').drop('Category_1', axis=1)

df9.head(2)

,HTMCP-03-06-02001,HTMCP-03-06-02002,HTMCP-03-06-02006,HTMCP-03-06-02007,HTMCP-03-06-02012,HTMCP-03-06-02020,HTMCP-03-06-02026,HTMCP-03-06-02036,HTMCP-03-06-02040,HTMCP-03-06-02042,...,HTMCP-03-06-02266,HTMCP-03-06-02267,HTMCP-03-06-02268,HTMCP-03-06-02320,HTMCP-03-06-02330,HTMCP-03-06-02332,HTMCP.03.06.02037,HTMCP.03.06.02046,HTMCP.03.06.02150,HTMCP.03.06.02174
SWI/SNF remodeling complex,0.0,0.0,0.0,5.0,0.0,0.0,2.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
SWI/SNF family,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [46]:
df10 = pd.concat([df8, df9]).reset_index().dropna().set_index('index').astype(float)

df10.head(2)

,HTMCP-03-06-02001,HTMCP-03-06-02002,HTMCP-03-06-02006,HTMCP-03-06-02007,HTMCP-03-06-02012,HTMCP-03-06-02020,HTMCP-03-06-02026,HTMCP-03-06-02036,HTMCP-03-06-02040,HTMCP-03-06-02042,...,HTMCP-03-06-02266,HTMCP-03-06-02267,HTMCP-03-06-02268,HTMCP-03-06-02320,HTMCP-03-06-02330,HTMCP-03-06-02332,HTMCP.03.06.02037,HTMCP.03.06.02046,HTMCP.03.06.02150,HTMCP.03.06.02174
index,,,,,,,,,,,,,,,,,,,,,
Remodeller,0.0,0.0,2.0,16.0,1.0,1.0,9.0,0.0,2.0,2.0,...,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
Reader,1.0,0.0,0.0,21.0,1.0,1.0,7.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0


In [47]:
df10[df10 < 0.5] = np.nan
df10[df10 > 0.5] = 'Positive'
df10.fillna('Negative', inplace=True)

In [48]:
df10.head(2)

,HTMCP-03-06-02001,HTMCP-03-06-02002,HTMCP-03-06-02006,HTMCP-03-06-02007,HTMCP-03-06-02012,HTMCP-03-06-02020,HTMCP-03-06-02026,HTMCP-03-06-02036,HTMCP-03-06-02040,HTMCP-03-06-02042,...,HTMCP-03-06-02266,HTMCP-03-06-02267,HTMCP-03-06-02268,HTMCP-03-06-02320,HTMCP-03-06-02330,HTMCP-03-06-02332,HTMCP.03.06.02037,HTMCP.03.06.02046,HTMCP.03.06.02150,HTMCP.03.06.02174
index,,,,,,,,,,,,,,,,,,,,,
Remodeller,Negative,Negative,Positive,Positive,Positive,Positive,Positive,Negative,Positive,Positive,...,Positive,Negative,Negative,Negative,Negative,Positive,Negative,Negative,Negative,Negative
Reader,Positive,Negative,Negative,Positive,Positive,Positive,Positive,Negative,Negative,Positive,...,Negative,Negative,Negative,Positive,Positive,Positive,Negative,Negative,Negative,Negative


In [49]:
category_track.head(2)

,patient,HIV_status,Putative_histology,HPV_status
0,HTMCP-03-06-02007,Negative,Adeno,Negative
1,HTMCP-03-06-02026,Negative,Adeno,Negative


In [50]:
df11 = df10.T

In [51]:
df11.head(2)

index,Remodeller,Reader,Writer,Eraser,SWI/SNF remodeling complex,SWI/SNF family,ISWI remdelling complexes,SEC (Super Elongation) Complex,INO80/SWR remodelling complexes,Testis-specific chromatin protein,...,ISWI chromatin remdelling,HAT complexes,NuRD complex,compass complex,HDAC complex,GNAT family HAT complexes,MYST family HAT complexes,RNF20/40 complex,SWI/SNF remodeling complex,TFIID basal transcription factor complex or PCAF histone acetylase complex
HTMCP-03-06-02001,Negative,Positive,Positive,Negative,Negative,Negative,Negative,Negative,Positive,Negative,...,Negative,Negative,Negative,Positive,Negative,Positive,Negative,Negative,Negative,Positive
HTMCP-03-06-02002,Negative,Negative,Positive,Negative,Negative,Negative,Negative,Negative,Negative,Negative,...,Negative,Negative,Negative,Positive,Negative,Negative,Negative,Negative,Negative,Negative


In [52]:
category_track = pd.merge(category_track, df11, left_on='patient', right_index=True, how='left')

In [56]:
of ='{}category_track.txt'.format(wkdir)
category_track.to_csv(of, index=False)

process chipseq signals

In [36]:
def get_files(f):
    # f = '/projects/trans_scratch/validations/workspace/szong/Cervical/chipseq/sorted_UCSC_A84611_H3K4me3_treat_pileup.bdg.intersected'
    df = pd.read_csv(f,header=None)
    df.columns = ['file']
    df.head(2)
    return df


In [46]:
def integrate_signals(f):
#     f = '/projects/trans_scratch/validations/workspace/szong/Cervical/chipseq/pileup_files/sorted_UCSC_A84611_H3K4me3_treat_pileup.bdg.intersected'
    df = pd.read_csv(f, sep='\t', header=None)
    df.columns = ['chr', 'start', 'end', 'gene', 'chip_chr', 'chip_start', 'chip_end', 'peak_height']
    df['area'] = df.peak_height * (df.chip_end - df.chip_start)
    dfg = df.groupby(['chr', 'start', 'end', 'gene'])['area'].sum().reset_index()
    of = '{0}.{1}'.format(f, 'sum')
    dfg.to_csv(of, index=False)

In [39]:

def process_signals(df1):
#     f1 = '/projects/trans_scratch/validations/workspace/szong/Cervical/chipseq/treat_pileup_files.txt' 
    for idx, f in df1.iterrows():
        f = '/projects/trans_scratch/validations/workspace/szong/Cervical/chipseq/H3K36me3_gb/{}'.format(f['file'])
        integrate_signals(f)

In [47]:
# f1 = '/projects/trans_scratch/validations/workspace/szong/Cervical/chipseq/pileup_files/treat_intersected_files.txt'
f1 = '/projects/trans_scratch/validations/workspace/szong/Cervical/chipseq/H3K36me3_gb/H3K36me3_treat_intersected_files.txt'
df1 = get_files(f1)
process_signals(df1)

In [48]:
df1.head(2)

,file
0,sorted_UCSC_A84614_H3K36me3_treat_pileup.bdg.i...
1,sorted_UCSC_A84621_H3K36me3_treat_pileup.bdg.i...


In [20]:
# # process control, sum signals
# f2 = '/projects/trans_scratch/validations/workspace/szong/Cervical/chipseq/control_files.txt'
# df2 = get_files(f2)
# # process_signals(f2)

In [21]:
# df3 = pd.merge(df1, df2, left_index=True, right_index=True)
# df3.head(2)

,file_x,file_y
0,sorted_UCSC_A84611_H3K4me3_treat_pileup.bdg,sorted_UCSC_A84611_H3K4me3_control_lambda.bdg
1,sorted_UCSC_A84618_H3K4me3_treat_pileup.bdg,sorted_UCSC_A84618_H3K4me3_control_lambda.bdg


In [24]:
def visulize_control_treat_signal():
    # scatter plot to visualize treat vs control signal 
    num_pats = df3.shape[0]
    plt_per_row = 8
    nrow = math.floor(num_pats/plt_per_row)+1

    fig, axes = plt.subplots(nrow, plt_per_row, sharex='col', sharey='row', figsize=(2*plt_per_row, nrow*2))
    for idx, f in df3.iterrows():
        f4 = '{0}{1}.intersected.sum'.format(wkdir, f['file_x'])
        library = f['file_x'].split('_')[2]
        f5 = '{0}{1}.intersected.sum'.format(wkdir, f['file_y'])
        df4 = pd.read_csv(f4).set_index(['chr', 'start', 'end', 'gene'])
        df5 = pd.read_csv(f5).set_index(['chr', 'start', 'end', 'gene'])
        dfm = pd.merge(df4, df5, left_index=True, right_index=True)
    #     print(dfm.head(2))
        axrow = math.floor(idx/plt_per_row)
        axcol = idx%plt_per_row
        ax = axes[axrow, axcol]
        sns.regplot(x = 'peak_height_y', y = 'peak_height_x',fit_reg=False, ax=ax, data = dfm)
        ax.set(xlabel='control', ylabel='treat')
        ax.set(xlim=(0, 200000), ylim=(0, 300000))
        ax.text(50000, 250000, library)
    plt.suptitle('H3K4me3_bdg_sum_control_vs_treat', fontsize=14)  
    plt.tight_layout()
    of = '{}H3K4me3_bdg_sum_control_vs_treat.png'.format(wkdir)
    plt.savefig(of, dpi=200, box_inches='tight')
    plt.show()


In [749]:
pdist?

merge signals from all libraries into matrix

In [66]:
df1.head(2)

,file
0,sorted_UCSC_A84610_H3K4me1_treat_pileup.bdg.in...
1,sorted_UCSC_A84611_H3K4me3_treat_pileup.bdg.in...


In [67]:
df1.shape

(210, 1)

In [50]:
print(df1.loc[1,:][0])

sorted_UCSC_A84621_H3K36me3_treat_pileup.bdg.intersected


In [68]:
marks = ["H3K4me1",
"H3K4me3",
"H3K9me3",
"H3K27me3",
"H3K36me3",
"H3K27ac"
]

In [37]:
# df1.file.str.contains('H3K4me3')

In [51]:
def merge_data(df):
    dfm = pd.DataFrame()
    for idx, f in df.iterrows():
        f1 = '{0}/{1}/{2}.sum'.format(wkdir, 'pileup_files', f['file'])
#         library = '_'.join(f['file_y'].split('_')[2:5])
        library = f['file'].split('_')[2]
        df1 = pd.read_csv(f1).set_index(['chr', 'start', 'end', 'gene'])
        df1.columns = [library]
        if dfm.empty:
            dfm = df1.copy()
        else:
            dfm = pd.merge(dfm, df1, left_index=True, right_index=True)
    return dfm
 

unsupervised clustering based on mean promoter H3K4me3 signal

heatmap to visualize the data

In [73]:
# cmap = sns.light_palette('red', as_cmap=True)
# g = sns.clustermap(dfm2, cmap = cmap)
# plt.setp(g.ax_heatmap.yaxis.get_majorticklabels(), rotation=0)
# pass

In [72]:
def make_dendrogram(dfmm2, mark):
#     dfmm2 = dfm.copy()
    np.isnan(dfmm2.values).any()
    np.isinf(dfmm2.values).any()

    # cluster by promoter signals so the array needs to be [number of patients, number of features]
    matrix = dfmm2.values.T
    matrix

    dist='correlation'
    link='ward'
    Z = linkage(pdist(matrix, dist), link)

    fig, ax = plt.subplots(figsize=(12,6))
    aa = dendrogram(Z, leaf_rotation=90.,labels = dfmm2.columns,leaf_font_size=12.,show_contracted=True)#
#     plt.suptitle('unsupervised_clustering_{}_bdg_mean'.format, fontsize=14)  
    plt.title('unsupervised_clustering_{}_bdg_mean'.format, fontsize=14)
    plt.tight_layout()
    of = '{0}unsupervised_clustering_{1}_bdg_mean.png'.format(wkdir, mark)
    plt.savefig(of, dpi=200, box_inches='tight')

In [74]:
# df_track.head()

In [81]:
# def get_track_df(dfm2, df_meta):
#     # get labels of the dendrogram and make track data frame
#     # cluster by promoter signals so the array needs to be [number of patients, number of features]
#     matrix = dfm2.values.T
#     dist='correlation'
#     link='ward'
#     Z = linkage(pdist(matrix, dist), link)
#     aa = dendrogram(Z, leaf_rotation=90.,labels = dfm2.columns,leaf_font_size=12.,show_contracted=True)
#     den_labels = aa['ivl']
#     adf = pd.DataFrame(den_labels)
#     adf.columns = ['patient']
#     df_track = pd.merge(adf, df_meta[['patient', 'HIV_status', 'Putative_histology' ]], left_on='patient', right_on='patient')#.drop(0, axis=1)
#     df_track = df_track.set_index('patient')
#     return (Z, df_track)

In [76]:
# dfm_mark.head(2)

In [77]:
df_meta.head(2)

,H3K4me1 library ID,H3K4me3 library ID,H3K9me3 library ID,H3K27me3 library ID,H3K36me3 library ID,H3K27ac library ID,input DNA control library ID,H3K4me1 total reads,H3K4me3 total reads,H3K9me3 total reads,H3K27me3 total reads,H3K36me3 total reads,H3K27ac total reads,input DNA control total reads,patient,HIV_status,Putative_histology,strelka_mutations,HPV_status
0,A84624,A84625,A84626,A84627,A84628,A84629,A84630,78611726,60179836,76557498,90312752.0,106610232.0,80676254.0,86539886.0,HTMCP-03-06-02007,Negative,Adeno,2958282,Negative
1,A84645,A84646,A84647,A84648,A84649,A84650,A84651,85566812,74487212,93042922,89559600.0,87905340.0,84986034.0,88575282.0,HTMCP-03-06-02026,Negative,Adeno,1252217,Negative


plot hiv and histology track

In [52]:
def map_features_to_colors(df_track, map_dict1, clrs):
#     clrs = ["purple", "#E69F00","#009E73","#56B4E9"]#,"#377EB8","#E41A1C","#F0E442",'black',"#D55E00","#CC79A7"]
#     map_dict1 =  {'Adeno': 0.15,'Squamous': 0.25, 'Negative': 0.35, 'Positive': 0.45}
    map_dict2 = {}
    for n, k in enumerate(map_dict1.keys()):
        map_dict2[k] = clrs[n]
    assert list(map_dict2.values()) == clrs
    ndf_track = df_track.apply(lambda x: x.map(map_dict1)).fillna(0).T
#     legends = sorted(map_dict1, key=map_dict1.get, reverse=False)
    return (ndf_track, map_dict2)

In [53]:
def plot_cluster_bk(numeric_track, category_track, dfm, mark, map_dict2): # df= tracks, dfm = histone mark signal df
#     df = ncategory_track.copy() # track df
    # dfm: histone mark signal df
    
    fig = plt.figure(figsize=(16, 4), facecolor='white')
    gs = gridspec.GridSpec(3, 2, height_ratios=[0.5,1.5,8], width_ratios=[17,1], 
                           wspace=0.05, # horizontal space
                           hspace=0.0  # vertical space
                          )

    bounds = np.arange(0.05, len(map_dict1)/10.0 + 0.1 , 0.1).tolist()
    assert len(bounds) == len(map_dict1) + 1
    # make a color map of fixed colors
    cmap = colors.ListedColormap(clrs)
    norm = colors.BoundaryNorm(bounds, cmap.N)
    nrow = len(category_track.index)
    ncol = len(category_track.columns)
    print(nrow, ncol)

    # add dendrogram
    ax5 = fig.add_subplot(gs[4], facecolor='white')
    matrix = dfm.values.T
    dist='correlation'
    link='ward'
    Z = linkage(pdist(matrix, dist), link)
    aa = dendrogram(Z, leaf_rotation=90.,labels = dfm.columns,leaf_font_size=12.,show_contracted=True)#
    den_labels = aa['ivl']
    adf = pd.DataFrame(den_labels)
    adf.columns = ['patient']
    c_track = pd.merge(adf, category_track.T, left_on='patient', right_index=True)#.drop(0, axis=1)
    c_track = c_track.set_index('patient')
    c_track = c_track.T
    plt.suptitle('unsupervised_clustering_{}_bdg_mean'.format(mark), fontsize=14)  
    print(den_labels)
    # plt.tight_layout()

    
    # plot category track
    ax3 = fig.add_subplot(gs[2])
    img = ax3.imshow(c_track.values, interpolation='none', origin='upper', aspect='auto',
                        cmap=cmap, norm=norm)
    _ = plt.yticks(np.arange(0, nrow, 1), c_track.index, fontsize=8)
    _ = plt.xticks(np.arange(0, ncol, 1), c_track.columns, fontsize=8, rotation=90)

    _ = ax3.set_yticks(np.arange(-.5, nrow, 1), minor=True);
    _ = ax3.set_xticks(np.arange(-.5, ncol, 1), minor=True);

    # Gridlines based on minor ticks
    _ = ax3.grid(b=True, which='minor', color='gray', linestyle='--', linewidth=0.3)
    # hide major tick grid
    _ = ax3.grid(b=True, which='major', color='r', linestyle='-', linewidth=0)

    ax3.spines['right'].set_visible(True)
    ax3.spines['top'].set_visible(True)
    ax3.set_xticklabels('')
    print(c_track.columns)
    
    
    # plot numeric track
    ax1 = fig.add_subplot(gs[0])
    numeric_track = numeric_track.set_index('patient').astype(float)
    n_track = pd.merge(adf, numeric_track, left_on='patient', right_index=True)#.drop(0, axis=1)
    n_track = n_track.set_index('patient')
    n_track = n_track.T
    nrow = len(n_track.index)
    ncol = len(n_track.columns)
    print(nrow, ncol)
    img = ax1.imshow(n_track.values, interpolation='none', origin='upper', aspect='auto', cmap='gray')
    _ = plt.yticks(np.arange(0, nrow, 1), n_track.index, fontsize=8)
    _ = plt.xticks(np.arange(0, ncol, 1), n_track.columns, fontsize=8, rotation=90)

    _ = ax1.set_yticks(np.arange(-.5, nrow, 1), minor=True);
    _ = ax1.set_xticks(np.arange(-.5, ncol, 1), minor=True);

    # Gridlines based on minor ticks
    _ = ax1.grid(b=True, which='minor', color='gray', linestyle='--', linewidth=0.3)
    # hide major tick grid
    _ = ax1.grid(b=True, which='major', color='r', linestyle='-', linewidth=0)

    ax1.spines['right'].set_visible(True)
    ax1.spines['top'].set_visible(True)
    ax1.set_xticklabels('')
    print(n_track.columns)
    

    # color bar for numeric_track
    ax2 = fig.add_subplot(gs[1])
    fig.colorbar(img)
    
    # make sure columns in numeric, category, and dendrogram match
    assert den_labels == c_track.columns.tolist()
    assert den_labels == n_track.columns.tolist()
    
    
    # add track legend
    ax6 = fig.add_subplot(gs[5], facecolor='white')
    # plot mutation legend
    k = 1
    for key in map_dict2:
        if key !=0: # don't plot key=0 which means no mutation
            clr = map_dict2[key]
            rect = patches.Rectangle((0,k/10), 0.3, 0.05, linewidth=0.01,edgecolor=clr,facecolor=clr)
            _ = ax6.text(0.35, k/10+.02, key, ha='left', va='center', fontsize=8)
            _ = ax6.add_patch(rect)
        k+=1
    ax6.get_xaxis().set_ticks([])
    ax6.get_yaxis().set_ticks([])

    ax6.set_title('')


    of = '/projects/trans_scratch/validations/workspace/szong/Cervical/chipseq/unsupervised_clustering_{}_bdg_mean.png'.format(mark)
    %time plt.savefig(of, bbox_inches='tight',dpi=300)
    plt.show()
    plt.close()

In [63]:
def plot_cluster(numeric_track, category_track, dfm, mark, map_dict2): # df= tracks, dfm = histone mark signal df
#     df = ncategory_track.copy() # track df
    # dfm: histone mark signal df
    
    fig = plt.figure(figsize=(16, 8), facecolor='white')
    gs = gridspec.GridSpec(1, 2, width_ratios=[17,1], 
                           wspace=0.05, # horizontal space
                           hspace=0.0  # vertical space
                          )

    bounds = np.arange(0.05, len(map_dict1)/10.0 + 0.1 , 0.1).tolist()
    assert len(bounds) == len(map_dict1) + 1
    # make a color map of fixed colors
    cmap = colors.ListedColormap(clrs)
    norm = colors.BoundaryNorm(bounds, cmap.N)
    nrow = len(category_track.index)
    ncol = len(category_track.columns)
    print(nrow, ncol)

    # left panel 3x1 for n_track, c_trak, and dendrogram
    gs01 = gridspec.GridSpecFromSubplotSpec(3, 1,  height_ratios=[.5, 14, 4], subplot_spec=gs[0],  hspace=0.0)
    
     # add dendrogram
    ax1_3 = fig.add_subplot(gs01[2])
    ax1_3.set_title("")
#     ax1_3 = fig.add_subplot(gs[4], facecolor='white')
    matrix = dfm.values.T
    dist='correlation'
    link='ward'
    Z = linkage(pdist(matrix, dist), link)
    aa = dendrogram(Z, leaf_rotation=90.,labels = dfm.columns,leaf_font_size=12.,show_contracted=True)#
    den_labels = aa['ivl']
    adf = pd.DataFrame(den_labels)
    adf.columns = ['patient']
    c_track = pd.merge(adf, category_track.T, left_on='patient', right_index=True)#.drop(0, axis=1)
    c_track = c_track.set_index('patient')
    c_track = c_track.T
    plt.suptitle('unsupervised_clustering_{}_bdg_area'.format(mark), fontsize=14)  
#     print(den_labels)
    

    
    
    # plot category track
    ax1_2 = fig.add_subplot(gs01[1], facecolor='white')
#     ax1_2.set_title("")
#     print(c_track)
    img = ax1_2.imshow(c_track.values, interpolation='none', origin='upper', aspect='auto',
                        cmap=cmap, norm=norm)
    _ = plt.yticks(np.arange(0, nrow, 1), c_track.index, fontsize=8)
    _ = plt.xticks(np.arange(0, ncol, 1), c_track.columns, fontsize=8, rotation=90)

    _ = ax1_2.set_yticks(np.arange(-.5, nrow, 1), minor=True);
    _ = ax1_2.set_xticks(np.arange(-.5, ncol, 1), minor=True);

    # Gridlines based on minor ticks
    _ = ax1_2.grid(b=True, which='minor', color='gray', linestyle='--', linewidth=0.3)
    # hide major tick grid
    _ = ax1_2.grid(b=True, which='major', color='r', linestyle='-', linewidth=0)

    ax1_2.spines['right'].set_visible(True)
    ax1_2.spines['top'].set_visible(True)
    ax1_2.set_xticklabels('')
#     print(c_track.columns)
    

    # plot numeric track
    ax1_1 = fig.add_subplot(gs01[0], facecolor='white')
#     ax1_1.set_title("")
    numeric_track = numeric_track.set_index('patient').astype(float)
    n_track = pd.merge(adf, numeric_track, left_on='patient', right_index=True)#.drop(0, axis=1)
    n_track = n_track.set_index('patient')
    n_track = n_track.T
    nrow = len(n_track.index)
    ncol = len(n_track.columns)
    print(nrow, ncol)
    img1_1 = ax1_1.imshow(n_track.values, interpolation='none', origin='upper', aspect='auto', cmap='Reds')
    _ = plt.yticks(np.arange(0, nrow, 1), n_track.index, fontsize=8)
    _ = plt.xticks(np.arange(0, ncol, 1), n_track.columns, fontsize=8, rotation=90)

    _ = ax1_1.set_yticks(np.arange(-.5, nrow, 1), minor=True);
    _ = ax1_1.set_xticks(np.arange(-.5, ncol, 1), minor=True);

    # Gridlines based on minor ticks
    _ = ax1_1.grid(b=True, which='minor', color='gray', linestyle='--', linewidth=0.3)
    # hide major tick grid
    _ = ax1_1.grid(b=True, which='major', color='r', linestyle='-', linewidth=0)

    ax1_1.spines['right'].set_visible(True)
    ax1_1.spines['top'].set_visible(True)
    ax1_1.set_xticklabels('')
#     print(n_track.columns)

    

    # right panel 2x1 for lengends, colorbar and rectangles
    gs02 = gridspec.GridSpecFromSubplotSpec(2, 1,  height_ratios=[1, 1], subplot_spec=gs[1])
    
    # color bar for numeric_track 
    ax2_1 = fig.add_subplot(gs02[0], facecolor='white')
    ax2_1.set_title("mutations_log10")
        
    _ = plt.colorbar(img1_1, cax=ax2_1, orientation= 'vertical', label='')#, aspect=40) 

    
    # add retangle legend for category track
    ax2_2 = fig.add_subplot(gs02[1], facecolor='white')
    ax2_2.set_title("rectangles")
        
#     ax6 = fig.add_subplot(gs[5], facecolor='white')
    # plot mutation legend
    k = 1
    for key in map_dict2:
        if key !=0: # don't plot key=0 which means no mutation
            clr = map_dict2[key]
            rect = patches.Rectangle((0,k/7), 0.3, 0.1, linewidth=0.01,edgecolor=clr,facecolor=clr)
            _ = ax2_2.text(0.35, k/7+.07, key, ha='left', va='center', fontsize=8)
            _ = ax2_2.add_patch(rect)
        k+=1
    ax2_2.get_xaxis().set_ticks([])
    ax2_2.get_yaxis().set_ticks([])

    ax2_2.set_title('')

  
    
    # make sure columns in numeric, category, and dendrogram match
#     print(den_labels)
#     print(c_track.columns.tolist())
    assert den_labels == c_track.columns.tolist()
    assert den_labels == n_track.columns.tolist()
#     plt.tight_layout()
    


#     of = '/projects/trans_scratch/validations/workspace/szong/Cervical/chipseq/unsupervised_clustering_{}_bdg_area.png'.format(mark)
    of = '/projects/trans_scratch/validations/workspace/szong/Cervical/chipseq/H3K36me3_gb/unsupervised_clustering_{}_bdg_area.png'.format(mark)
    %time plt.savefig(of, bbox_inches='tight',dpi=300)
    plt.show()
    plt.close()

In [54]:
category_track.head(2)

,patient,HIV_status,Putative_histology,HPV_status,Remodeller,Reader,Writer,Eraser,SWI/SNF remodeling complex,SWI/SNF family,...,ISWI chromatin remdelling,HAT complexes,NuRD complex,compass complex,HDAC complex,GNAT family HAT complexes,MYST family HAT complexes,RNF20/40 complex,SWI/SNF remodeling complex,TFIID basal transcription factor complex or PCAF histone acetylase complex
0,HTMCP-03-06-02007,Negative,Adeno,Negative,Positive,Positive,Positive,Positive,Positive,Positive,...,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive
1,HTMCP-03-06-02026,Negative,Adeno,Negative,Positive,Positive,Positive,Positive,Positive,Positive,...,Negative,Positive,Positive,Positive,Positive,Positive,Positive,Negative,Negative,Positive


In [57]:
numeric_track.head(2)

,patient,log_strelka_mutations
0,HTMCP-03-06-02007,6.47104
1,HTMCP-03-06-02026,6.09768


In [58]:
for mark in ["H3K36me3"]:#marks:
    print('merging signal for mark: {}'.format(mark))
    df_mark = df1[df1['file'].str.contains(mark)]
    dfm_mark = merge_data(df_mark)
    dfm_mark = dfm_mark/4000 # length of the promoter regaionx
    lib_name = '{} library ID'.format(mark)
    df_meta1 = df_meta[[lib_name, 'patient']].set_index(lib_name)
    d = df_meta1.to_dict()
    dfm_mark.columns = dfm_mark.columns.to_series().map(d['patient'])
    cat_track = category_track.set_index('patient')
    clrs = ["purple", "#E69F00","#009E73","#56B4E9"]#,"#377EB8","#E41A1C","#F0E442",'black',"#D55E00","#CC79A7"]
    map_dict1 =  {'Adeno': 0.15,'Squamous': 0.25, 'Negative': 0.35, 'Positive': 0.45}
    ncategory_track, map_dict2 = map_features_to_colors(cat_track, map_dict1, clrs)
    plot_cluster(numeric_track, ncategory_track, dfm_mark, mark, map_dict2)


merging signal for mark: H3K36me3


NameError: name 'df1' is not defined

In [61]:
dfm_mark.head(2)

HTMCP-03-06-02001  HTMCP-03-06-02006  \
chr  start end   gene                                                    
chr1 34081 38080 ENSG00000237613            2.96911           0.593800   
     67092 71091 ENSG00000186092            3.51677           0.584748   

                                  HTMCP-03-06-02007  HTMCP-03-06-02012  \
chr  start end   gene                                                    
chr1 34081 38080 ENSG00000237613           0.517766           0.572157   
     67092 71091 ENSG00000186092           0.690354           0.289419   

                                  HTMCP-03-06-02020  HTMCP-03-06-02026  \
chr  start end   gene                                                    
chr1 34081 38080 ENSG00000237613            1.86325            0.99825   
     67092 71091 ENSG00000186092            2.39625            0.21825   

                                  HTMCP-03-06-02040  HTMCP-03-06-02042  \
chr  start end   gene                                                    
chr1 34081 38080 ENSG00000237613           1.511414           1.577847   
     67092 71091 ENSG00000186092           0.951870           3.199019   

                                  HTMCP-03-06-02054  HTMCP-03-06-02058  \
chr  start end   gene                                                    
chr1 34081 38080 ENSG00000237613           1.154083           0.988018   
     67092 71091 ENSG00000186092           0.164869           0.982336   

                                        ...          HTMCP-03-06-02210  \
chr  start end   gene                   ...                              
chr1 34081 38080 ENSG00000237613        ...                    1.62525   
     67092 71091 ENSG00000186092        ...                    2.81525   

                                  HTMCP-03-06-02214  HTMCP-03-06-02202  \
chr  start end   gene                                                    
chr1 34081 38080 ENSG00000237613            1.92225           2.814546   
     67092 71091 ENSG00000186092            2.03700           2.040106   

                                  HTMCP-03-06-02182  HTMCP-03-06-02260  \
chr  start end   gene                                                    
chr1 34081 38080 ENSG00000237613           1.242604              1.888   
     67092 71091 ENSG00000186092           0.333469              1.744   

                                  HTMCP-03-06-02266  HTMCP-03-06-02268  \
chr  start end   gene                                                    
chr1 34081 38080 ENSG00000237613            1.39125            0.81125   
     67092 71091 ENSG00000186092            0.84800            0.87200   

                                  HTMCP-03-06-02320  HTMCP-03-06-02330  \
chr  start end   gene                                                    
chr1 34081 38080 ENSG00000237613             1.0790           1.003871   
     67092 71091 ENSG00000186092             0.4425           1.005059   

                                  HTMCP-03-06-02332  
chr  start end   gene                                
chr1 34081 38080 ENSG00000237613           1.423199  
     67092 71091 ENSG00000186092           2.165821  

[2 rows x 35 columns]

In [62]:
dfm_mark.shape

(20059, 35)

In [100]:
matplotlib.is_interactive()

True

In [236]:
df32 = pd.DataFrame(df3.file_y)
dfm2 = merge_data(df32)
dfm2 = dfm2/4000 # length of the promoter regaion
dfm2.head()

A84611_H3K4me3_control  \
chr  start  end    gene                                      
chr1 34081  38080  ENSG00000237613                0.427685   
     67092  71091  ENSG00000186092                0.061214   
     365641 369640 ENSG00000235249                0.041954   
     620053 624052 ENSG00000185097                0.043833   
     858261 862260 ENSG00000187634                0.559055   

                                    A84618_H3K4me3_control  \
chr  start  end    gene                                      
chr1 34081  38080  ENSG00000237613                0.310419   
     67092  71091  ENSG00000186092                0.055682   
     365641 369640 ENSG00000235249                0.009753   
     620053 624052 ENSG00000185097                0.019739   
     858261 862260 ENSG00000187634                0.287142   

                                    A84625_H3K4me3_control  \
chr  start  end    gene                                      
chr1 34081  38080  ENSG00000237613                0.252474   
     67092  71091  ENSG00000186092                0.113265   
     365641 369640 ENSG00000235249                0.007909   
     620053 624052 ENSG00000185097                0.003780   
     858261 862260 ENSG00000187634                0.111270   

                                    A84632_H3K4me3_control  \
chr  start  end    gene                                      
chr1 34081  38080  ENSG00000237613                0.204968   
     67092  71091  ENSG00000186092                0.049366   
     365641 369640 ENSG00000235249                0.034107   
     620053 624052 ENSG00000185097                0.044374   
     858261 862260 ENSG00000187634                0.226632   

                                    A84639_H3K4me3_control  \
chr  start  end    gene                                      
chr1 34081  38080  ENSG00000237613                0.228736   
     67092  71091  ENSG00000186092                0.203678   
     365641 369640 ENSG00000235249                0.041958   
     620053 624052 ENSG00000185097                0.027462   
     858261 862260 ENSG00000187634                0.014630   

                                    A84646_H3K4me3_control  \
chr  start  end    gene                                      
chr1 34081  38080  ENSG00000237613                0.146941   
     67092  71091  ENSG00000186092                0.133603   
     365641 369640 ENSG00000235249                0.016568   
     620053 624052 ENSG00000185097                0.020352   
     858261 862260 ENSG00000187634                0.094569   

                                    A84653_H3K4me3_control  \
chr  start  end    gene                                      
chr1 34081  38080  ENSG00000237613                0.130224   
     67092  71091  ENSG00000186092                0.105407   
     365641 369640 ENSG00000235249                0.022140   
     620053 624052 ENSG00000185097                0.058604   
     858261 862260 ENSG00000187634                0.020124   

                                    A84660_H3K4me3_control  \
chr  start  end    gene                                      
chr1 34081  38080  ENSG00000237613                0.237918   
     67092  71091  ENSG00000186092                0.316299   
     365641 369640 ENSG00000235249                0.013750   
     620053 624052 ENSG00000185097                0.044327   
     858261 862260 ENSG00000187634                0.032539   

                                    A84667_H3K4me3_control  \
chr  start  end    gene                                      
chr1 34081  38080  ENSG00000237613                0.352411   
     67092  71091  ENSG00000186092                0.073475   
     365641 369640 ENSG00000235249                0.018340   
     620053 624052 ENSG00000185097                0.017215   
     858261 862260 ENSG00000187634                0.262516   

                                    A84674_H3K4me3_control  \
chr  start  end    gene                                    

make input file for ChromHMM

In [91]:
f = '/home/szong/projects/META_2018/Master_Sample_table_14X292_Feb27_2018_sz.xls'
meta = pd.read_excel(f, sheetname='HTMCP CaCx epi')
meta.head(2)

,Order,Biospecimen ID,HIV status,Epigenome modifier mutation status,H3K4me1 library ID,H3K4me3 library ID,H3K9me3 library ID,H3K27me3 library ID,H3K36me3 library ID,H3K27ac library ID,...,H3K4me3 % aligned,H3K9me3 % aligned,H3K27me3 % aligned,H3K36me3 % aligned,H3K27ac % aligned,input DNA control % aligned,internal sample ID,pool ID,Notes,# lanes 7 PE
0,1,HTMCP-03-06-02007-01A,negative,mutated,A84624,A84625,A84626,A84627,A84628,A84629,...,98.13,93.1,97.93,98.57,98.10,96.26,NaN,IX5978,NaN,1
1,2,HTMCP-03-06-02026-01A,negative,mutated,A84645,A84646,A84647,A84648,A84649,A84650,...,97.68,93.89,97.84,98.17,97.86,95.56,NaN,IX5981,NaN,1


In [92]:
meta.columns

Index(['Order', 'Biospecimen ID', 'HIV status',
       'Epigenome modifier mutation status', 'H3K4me1 library ID',
       'H3K4me3 library ID', 'H3K9me3 library ID', 'H3K27me3 library ID',
       'H3K36me3 library ID', 'H3K27ac library ID',
       'input DNA control  library ID', ' Library construction status',
       'Sequencing status', 'H3K4me1 total reads', 'H3K4me3  total reads',
       'H3K9me3 total reads', 'H3K27me3 total reads', 'H3K36me3 total reads',
       'H3K27ac total reads', 'input DNA control  total reads',
       'H3K4me1 % aligned', 'H3K4me3  % aligned', 'H3K9me3 % aligned',
       'H3K27me3 % aligned', 'H3K36me3 % aligned', 'H3K27ac % aligned',
       'input DNA control   % aligned', 'internal sample ID', 'pool ID',
       'Notes', '# lanes 7 PE'],
      dtype='object')

In [93]:
meta.set_index('Biospecimen ID', inplace=True)

In [94]:
df = meta[['H3K4me1 library ID',
       'H3K4me3 library ID', 'H3K9me3 library ID', 'H3K27me3 library ID',
       'H3K36me3 library ID', 'H3K27ac library ID']].stack().reset_index()

In [95]:
df.head(2)

,Biospecimen ID,level_1,0
0,HTMCP-03-06-02007-01A,H3K4me1 library ID,A84624
1,HTMCP-03-06-02007-01A,H3K4me3 library ID,A84625


In [96]:
df['marks'] = df.level_1.apply(lambda x: x.replace(' library ID', ''))
df['patient'] = df['Biospecimen ID'].apply(lambda x: x.replace('-01A', ''))
df.drop('Biospecimen ID', axis=1, inplace=True)

In [97]:
df = df.drop('level_1', axis=1)

In [98]:
df.head(2)

,0,marks,patient
0,A84624,H3K4me1,HTMCP-03-06-02007
1,A84625,H3K4me3,HTMCP-03-06-02007


In [100]:
f = '/projects/da_workspace/software/ChromHMM/Cervical/libs.txt'
df.to_csv(f, sep='\t', index=False, header=False)

In [81]:
df['treat'] = 'sorted_UCSC_' + df[0] + '_' + df['marks'] + '_treat_pileup.bdg'

In [82]:
df['control'] = 'sorted_UCSC_' + df[0] + '_' + df['marks'] + '_control_lambda.bdg'

In [83]:
df.head(2)

,0,marks,patient,treat,control
0,A84624,H3K4me1,HTMCP-03-06-02007,sorted_UCSC_A84624_H3K4me1_treat_pileup.bdg,sorted_UCSC_A84624_H3K4me1_control_lambda.bdg
1,A84625,H3K4me3,HTMCP-03-06-02007,sorted_UCSC_A84625_H3K4me3_treat_pileup.bdg,sorted_UCSC_A84625_H3K4me3_control_lambda.bdg


In [84]:
df = df.drop(0, axis=1)

In [85]:
df.head(2)

,marks,patient,treat,control
0,H3K4me1,HTMCP-03-06-02007,sorted_UCSC_A84624_H3K4me1_treat_pileup.bdg,sorted_UCSC_A84624_H3K4me1_control_lambda.bdg
1,H3K4me3,HTMCP-03-06-02007,sorted_UCSC_A84625_H3K4me3_treat_pileup.bdg,sorted_UCSC_A84625_H3K4me3_control_lambda.bdg


In [87]:
cols = ['patient', 'marks', 'treat', 'control']

In [89]:
df = df[cols]

In [90]:
f = '/projects/da_workspace/software/ChromHMM/Cervical/bed_files.txt'
df.to_csv(f, sep='\t', index=False, header=False)

In [106]:
df1 = df.set_index(0)

In [104]:
fbam = '/projects/da_workspace/software/ChromHMM/Cervical/bam_files.txt'
dfbam = pd.read_csv(fbam, sep='\t', header=None, index_col=0)
dfbam.head(2)

,1
0,
A84610,/projects/analysis/analysis30/IX5976/CC0TLANXX...
A84611,/projects/analysis/analysis30/IX5976/CC0TLANXX...


In [109]:
f = '/projects/da_workspace/software/ChromHMM/Cervical/bam_files.txt'
pd.merge(df1, dfbam, left_index=True, right_index=True).set_index('patient', drop=True).to_csv(f, sep='\t', header=False)

In [3]:

f = '/projects/da_workspace/software/ChromHMM/Cervical/signal_intervals.txt'
df = pd.read_csv(f)
df.head()

/home/szong/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,H3K27ac,H3K27me3,H3K36me3,H3K4me1,H3K4me3,H3K9me3,chr,ix1,ix2
0,0,0,0,0,0,0,1,1,65200,65400
1,1,0,1,0,0,0,0,1,71200,71400
2,2,1,0,0,0,0,0,1,71400,71600
3,3,0,0,0,0,0,1,1,72400,72600
4,4,0,1,0,0,0,0,1,83000,83200


In [5]:
df.columns

Index(['Unnamed: 0', 'H3K27ac', 'H3K27me3', 'H3K36me3', 'H3K4me1', 'H3K4me3',
       'H3K9me3', 'chr', 'ix1', 'ix2'],
      dtype='object')

H3K27ac     1730476
H3K27me3    1309242
H3K36me3    2575782
H3K4me1     2247894
H3K4me3     1058464
H3K9me3      980628
dtype: int64